In [1]:
import pandas as pd
import numpy as np
import findspark
import re
from functools import reduce
from pyspark import SparkContext
from pyspark.sql import SparkSession, Window, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
from matplotlib import pyplot as plt

pd.set_option("display.max_columns", None)

In [2]:
# create entry
spark = SparkSession.builder.appName("MarchMadness").getOrCreate()

def load_dataframe(file_path):
    """read file from relative path."""
    df = spark.read.option('header', True).format('csv').load(file_path)
    return df

### Basic Modeling

In [65]:
season_agg = load_dataframe('./write_data/season_agg.csv').drop('Name')

In [54]:
file_path = '../MarchMadness2021/data/MNCAATourneyCompactResults.csv'
tourney = load_dataframe(file_path).withColumn('Target', col('WScore') - col('LScore'))
tourney.limit(5).toPandas()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Target
0,1985,136,1116,63,1234,54,N,0,9.0
1,1985,136,1120,59,1345,58,N,0,1.0
2,1985,136,1207,68,1250,43,N,0,25.0
3,1985,136,1229,58,1425,55,N,0,3.0
4,1985,136,1242,49,1325,38,N,0,11.0


In [4]:
tourneyseeds = load_dataframe('./data/MNCAATourneySeeds.csv') 
tourneyseeds.limit(5).toPandas()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [7]:
# regular season features with tournament season seeding set up
regular_with_seed = season_agg.select('Season', col('TeamID').alias('WTeamID')
                                      , col('Pct').cast(IntegerType()).alias('WPct')
                                     , col('rank').cast(IntegerType()).alias('WRank')) \
                              .join(tourney, ['Season', 'WTeamID'], 'inner') \
                              .join(tourneyseeds.select('Season', col('Seed').alias('WSeed')
                                                        , col('TeamID').alias('WTeamID')) \
                                    , ['Season', 'WTeamID'], 'inner') \
                              .join(season_agg.select('Season', col('TeamID').alias('LTeamID')
                                                      , col('Pct').cast(IntegerType()).alias('LPct')
                                                      , col('rank').cast(IntegerType()).alias('LRank')) \
                                     , ['Season', 'LTeamID'], 'inner') \
                              .join(tourneyseeds.select('Season', col('Seed').alias('LSeed')
                                                        , col('TeamID').alias('LTeamID'))
                                   , ['Season', 'LTeamID'], 'inner') 
    
#                  .withColumn('LSeed#', concat(tourney.LSeed.substr(2, 2)).cast(IntegerType())) \
#                  .withColumn('WSeed#', concat(tourney.WSeed.substr(2, 2)).cast(IntegerType())) \


regular_with_seed = regular_with_seed \
                 .withColumn('LSeed#', concat(regular_with_seed.LSeed.substr(2, 2)).cast(IntegerType())) \
                 .withColumn('WSeed#', concat(regular_with_seed.WSeed.substr(2, 2)).cast(IntegerType())) \
                    .select('Season', 'LTeamID', 'WTeamID'
                            , col("WScore").cast(IntegerType()), col("LScore").cast(IntegerType())
                            , 'WPct', 'LPct', 'WRank', 'LRank', 'LSeed#', 'WSeed#')
regular_with_seed.limit(5).toPandas()

,Season,LTeamID,WTeamID,WScore,LScore,WPct,LPct,WRank,LRank,LSeed#,WSeed#
0,2015,1459,1116,56,53,76,81,22,80,12,5
1,2004,1150,1403,76,73,68,72,29,34,9,8
2,2006,1393,1401,66,58,72,67,36,26,5,12
3,2007,1172,1268,82,70,75,87,14,46,13,4
4,2010,1196,1140,99,92,84,63,14,48,10,7


In [8]:
# lets make a model based on Pct
regular_with_seed = regular_with_seed \
                     .withColumn('pred_bypct', when(col('WPct') > col('LPct'), 1).otherwise(0)) \
                     .withColumn('pred_byseed', when(col('WSeed#') < col('LSeed#'), 1).otherwise(0)) \
                     .withColumn('pred_byrank', when(col('WRank') < col('LRank'), 1).otherwise(0))

In [9]:
regular_with_seed.select('pred_bypct', 'pred_byseed', 'pred_byrank') \
                .groupby().agg(sum(lit(1)).alias('Actual')
                         ,sum('pred_bypct').alias('PredPct')
                         , sum('pred_byseed').alias('PredSeed')
                         , sum('pred_byrank').alias('PredRank')) \
                .withColumn('PredPct_', round(100*col('PredPct')/col('Actual'), 2)) \
                .withColumn('PredSeed_', round(100*col('PredSeed')/col('Actual'), 2)) \
                .withColumn('PredRank_', round(100*col('PredRank')/col('Actual'), 2)) \
                .show()              

+------+-------+--------+--------+--------+---------+---------+
|Actual|PredPct|PredSeed|PredRank|PredPct_|PredSeed_|PredRank_|
+------+-------+--------+--------+--------+---------+---------+
|  1108|    677|     750|     786|    61.1|    67.69|    70.94|
+------+-------+--------+--------+--------+---------+---------+



* Predicting each match based rank average has better predictability than seed and then winning percentage.
* Average for each match should give us a higher probability than 61%.
* Seed is not available for the first prediction round of submission. 

In [51]:
# lets create the data for training model
# data is since 2003
def map_columns(
    dataframe, original_col, map_win, map_loss, map_win_ex=None, map_loss_ex=None
):
    map_win = [column for column in map_win if column not in map_win_ex]
    map_loss = [column for column in map_loss if column not in map_loss_ex]

    original_col = [
        column
        for column in original_col
        if column not in ["TeamID", "TeamName", "Season"]
    ]
    zip_cols = zip(map_win, map_loss, original_col)
    for map_from, map_to, org_col in zip_cols:
        dataframe = (
            dataframe.withColumn(org_col, col(map_from) - col(map_to))
            .drop(map_from)
            .drop(map_to)
        )

    return dataframe


In [84]:
tourney_lr = tourney.withColumn('Target', col('WScore') - col('LScore')) \
                      .select('Season', 'WTeamID', 'LTeamID', 'Target') \

old_cols = [column for column in season_agg.columns if column != 'Season']
new_cols_L = ['L' + column for column in season_agg.columns if column != 'Season']
new_cols_W = ['W' + column for column in season_agg.columns if column != 'Season']

# rename columns
season_agg_L = reduce(lambda season_agg, idx: 
                   season_agg.withColumnRenamed(old_cols[idx], new_cols_L[idx])
                   , range(len(old_cols)), season_agg)

season_agg_L = season_agg_L.join(tourney_lr, ['LTeamID', 'Season'], 'inner')

# rename columns
season_agg_W = reduce(lambda season_agg, idx: 
                   season_agg.withColumnRenamed(old_cols[idx], new_cols_W[idx])
                   , range(len(old_cols)), season_agg)

season_agg_W = season_agg_W.join(tourney_lr, ['WTeamID', 'Season'], 'inner')

season_agg_lr = season_agg_W.join(season_agg_L
                                  , ['WTeamID', 'LTeamID', 'Season', 'Target']
                                  , 'inner')

map_win_ex = ['WTeamID', 'Season', 'Target']
map_loss_ex = ['LTeamID', 'Season', 'Target']
data_target_positive = map_columns(season_agg_lr, old_cols, new_cols_W, new_cols_L
                                   , map_win_ex=map_win_ex, map_loss_ex=map_loss_ex) 

In [85]:
data_target_negative = map_columns(season_agg_lr, old_cols, new_cols_L, new_cols_W
                   , map_win_ex=['LTeamID', 'Season', 'Target']
                   , map_loss_ex=['WTeamID', 'Season', 'Target']) \
                  .drop('WName', 'LName') \
                  .withColumn('Target', -col('Target'))

data = data_target_positive.union(data_target_negative)

In [86]:
data.withColumn('Target', when(col('Target') < 0, 0).otherwise(1)) \
    .toPandas().to_csv('./write_data/modeling_data.csv', header=True, index=False, float_format='%.1f')

In [89]:
# training and testing data
model_data = data.withColumn('win_loss', when(col('Target') < 0, 0).otherwise(1)).toPandas()
model_data['Season'] = pd.to_numeric(model_data['Season'])

training = model_data[model_data['Season'] < 2015]
testing = model_data[model_data['Season'] >= 2015]

In [114]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, cross_val_score
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, RandomForestClassifier, RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_absolute_error


In [147]:
def pipe_score(model, y_label = None, cv=None, scoring=None):
    # lets start with linear regression
    X_train = training.drop(columns=['win_loss', 'Target'])
    y_train = training[y_label].values.ravel()
    imp = SimpleImputer(strategy='mean')
    feat_selector = SelectKBest(f_regression, k=10)
    pipe = Pipeline(steps=[('imp', imp), ('select', feat_selector), ('model', model)])
    scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring=scoring)
    return scores


In [149]:
# lets start with linear regression
model = LinearRegression()
cv = KFold(n_splits=5)
pipe_score(model, y_label='Target', cv=cv, scoring='neg_mean_absolute_error')

array([ -9.64960871, -10.10377538,  -8.48031173, -10.26516469,
        -9.86017695])

In [116]:
# average MAE of the five folds
-1*scores.mean()

9.766326676592694

In [120]:
# the average difference in the scores between winning and losing teams in the target data
# prediction average doesn't seem that far off even though it the game can be decided by a single point
y_train[y_train > 0].mean()

11.41388174807198

In [150]:
# lets try simple logisitc regression
model = LogisticRegression(solver='liblinear', max_iter=1000)
cv = StratifiedKFold(n_splits=5)
pipe_score(model, y_label='win_loss', cv=cv, scoring='roc_auc')

array([0.82930638, 0.78010753, 0.72473118, 0.79693962, 0.7742349 ])

In [122]:
# from sklearn.metrics import SCORERS
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [156]:
# submission
df_sub = load_dataframe('./Data/MSampleSubmissionStage1.csv')
df_sub = df_sub \
                .withColumn('Season', split(df_sub.ID, '_').getItem(0)) \
                .withColumn('Team1', split(df_sub.ID, '_').getItem(1)) \
                .withColumn('Team2', split(df_sub.ID, '_').getItem(2))

df_sub.limit(5).toPandas()

,ID,Pred,Season,Team1,Team2
0,2015_1107_1112,0.5,2015,1107,1112
1,2015_1107_1116,0.5,2015,1107,1116
2,2015_1107_1124,0.5,2015,1107,1124
3,2015_1107_1125,0.5,2015,1107,1125
4,2015_1107_1129,0.5,2015,1107,1129


In [155]:
# tourney_lr = tourney.withColumn('Target', col('WScore') - col('LScore')) \
#                       .select('Season', 'WTeamID', 'LTeamID') \

old_cols = [column for column in season_agg.columns if column != 'Season']
new_cols_L = ['L' + column for column in season_agg.columns if column != 'Season']
new_cols_W = ['W' + column for column in season_agg.columns if column != 'Season']

# rename columns
season_agg_L = reduce(lambda season_agg, idx: 
                   season_agg.withColumnRenamed(old_cols[idx], new_cols_L[idx])
                   , range(len(old_cols)), season_agg)

season_agg_L = season_agg_L.join(tourney_lr, ['LTeamID', 'Season'], 'inner')

# rename columns
season_agg_W = reduce(lambda season_agg, idx: 
                   season_agg.withColumnRenamed(old_cols[idx], new_cols_W[idx])
                   , range(len(old_cols)), season_agg)

season_agg_W = season_agg_W.join(tourney_lr, ['WTeamID', 'Season'], 'inner')

season_agg_lr = season_agg_W.join(season_agg_L
                                  , ['WTeamID', 'LTeamID', 'Season', 'Target']
                                  , 'inner')

map_win_ex = ['WTeamID', 'Season', 'Target']
map_loss_ex = ['LTeamID', 'Season', 'Target']
data_target_positive = map_columns(season_agg_lr, old_cols, new_cols_W, new_cols_L
                                   , map_win_ex=map_win_ex, map_loss_ex=map_loss_ex) 

,Season,TeamID,Years_D1,GP,PTS,PTSDiff,Pct,FG,%FG,FG3,%FG3,FT,%FT,OR,DR,DR/OR,AST,TO,AST/TO,STL,STL/TO,BLK,PF,HomePTS,AwayPTS,NeutralPTS,HomeDiff,AwayDiff,NeutralDiff,HomePct,AwayPct,NeutralPct,HomeFG,AwayFG,NeutralFG,Home%FG,Away%FG,Neutral%FG,HomeFG3,AwayFG3,NeutralFG3,Home%FG3,Away%FG3,Neutral%FG3,HomeFT,AwayFT,NeutralFT,Home%FT,Away%FT,Neutral%FT,HomeOR,AwayOR,NeutralOR,HomeDR,AwayDR,NeutralDR,HomeDR/OR,AwayDR/OR,NeutralDR/OR,HomeAst,AwayAst,NeutralAst,HomeTO,AwayTO,NeutralTO,HomeSTL,AwaySTL,NeutralSTL,HomeSTL/TO,AwaySTL/TO,NeutralSTL/TO,HomeBlk,AwayBlk,NeutralBlk,HomePF,AwayPF,NeutralPF,rank
0,2003,1119,37.0,25,53.68,-13.8,12.0,41.18,36.51,35.59,10.95,67.61,16.02,7.92,21.56,2.72,11.72,17.36,0.68,5.56,0.32,1.6,19.32,44.0,68.6,40.0,-9.28,-3.8,-0.72,14.29,10.0,0.0,39.91,43.4,36.59,20.19,15.2,1.12,33.19,40.0,26.67,5.89,4.77,0.3,67.69,68.36,54.55,6.56,9.02,0.45,3.64,4.16,0.12,9.08,11.48,1.0,2.49,2.76,8.33,9.29,15.0,13.0,13.0,23.5,17.0,5.71,5.4,5.0,0.44,0.23,0.29,1.07,2.1,4.0,14.93,25.7,17.0,318
1,2003,1308,37.0,28,73.11,5.61,67.86,44.75,35.17,34.68,7.96,73.75,21.69,12.32,23.57,1.91,12.46,13.43,0.93,6.61,0.49,4.04,16.36,56.76,110.13,67.0,2.89,2.36,0.36,64.71,75.0,66.67,43.97,47.61,41.92,21.54,10.21,3.42,31.25,40.43,39.02,4.4,2.78,0.78,74.16,72.65,76.27,10.8,8.7,2.2,5.64,5.71,0.96,11.07,9.71,2.79,1.96,1.7,2.89,10.12,17.13,13.33,9.59,21.75,13.0,7.06,6.25,5.0,0.74,0.29,0.38,3.24,6.5,2.0,11.47,27.88,13.33,92
2,2003,1330,37.0,27,66.63,2.04,44.44,40.22,34.85,31.92,8.78,70.24,21.51,13.96,23.85,1.71,14.22,13.3,1.07,8.26,0.62,2.78,18.33,60.69,77.6,52.0,1.85,0.52,-0.33,50.0,40.0,0.0,40.29,40.72,33.33,19.96,13.9,1.0,30.42,34.74,26.32,4.84,3.67,0.28,68.42,74.67,50.0,11.56,9.34,0.61,7.56,5.96,0.44,13.0,9.93,0.93,1.72,1.66,2.08,14.38,14.3,11.0,11.0,17.1,12.0,9.13,7.7,0.0,0.83,0.45,0.0,3.06,2.3,3.0,15.38,22.6,23.0,187
3,2005,1186,37.0,27,61.41,-6.26,25.93,41.91,35.46,33.27,10.07,66.04,19.0,9.37,21.41,2.28,13.67,16.26,0.84,5.19,0.32,1.7,20.85,35.82,114.0,68.5,-4.96,-1.0,-0.3,23.53,25.0,50.0,41.55,40.67,51.09,22.38,10.25,2.83,32.02,32.59,47.22,6.39,2.65,1.03,61.58,70.2,61.9,7.06,10.37,1.57,4.0,4.7,0.67,8.37,11.78,1.26,2.09,2.5,1.89,8.0,25.13,16.0,8.76,33.13,12.5,5.65,4.38,4.5,0.64,0.13,0.36,1.29,3.0,0.0,11.59,40.75,20.0,276
4,2005,1216,37.0,28,59.46,-10.32,28.57,39.15,38.02,28.79,7.75,62.94,16.22,12.75,21.5,1.69,8.96,14.61,0.61,6.71,0.46,1.93,21.07,39.0,138.2,64.67,-6.79,-2.79,-0.75,35.0,0.0,33.33,39.59,35.16,43.35,26.97,6.55,4.5,30.16,22.73,31.11,5.71,1.2,0.84,70.0,56.61,60.0,7.99,6.43,1.8,5.5,6.11,1.14,9.61,9.39,2.5,1.75,1.54,2.19,5.55,21.2,11.33,8.05,40.2,15.7,6.55,7.4,6.7,0.81,0.18,0.43,1.3,5.0,1.0,12.2,58.2,18.33,297
